<a href="https://colab.research.google.com/github/VijayTaori/Lifo_step1/blob/main/Detailed_lifo_step_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import calendar
df_open = pd.read_csv('/content/et_open_alldata.csv')
df_close = pd.read_csv('/content/et_close_alldata.csv')
df_close['Close Datetime'] = pd.to_datetime(df_close['Close_unix_timestamp'],dayfirst=True,unit='s')
df_open['Open Datetime'] = pd.to_datetime(df_open['Open_unix_timestamp'],dayfirst=True,unit='s')
df_close_btc = df_close[df_close['Close Action'] =='Buy Bitcoin']
df_open_btc = df_open[df_open['Open Action'] =='Buy Bitcoin']
df_close_btc.reset_index(inplace=True)
df_close_btc = df_close_btc.drop('index', axis=1)
df_close_btc.reset_index(inplace=True)
df_close_btc.rename(columns={'index':'Original_Close_index'},inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc = df_open_btc.drop('index', axis=1)
df_open_btc.reset_index(inplace=True)
df_open_btc.rename(columns={'index':'Original_Open_index'},inplace=True)
data_btc = {}
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Original_Open_index','Original_Close_index',
                                    'Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units','Spread_Eur',
                                    'Open Amount_Eur','Amount_close_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur', 'Close Rate_Eur',
                                    'Open Action', 'Close Action', 
                                    'Open Amount','Close Rate', 'Open Rate',
                                    'Close Spread', 'Close Profit USD',
                                    'Close_Date','Open_Date',
                                    'Close_unix_timestamp','EurOpen_y', 
                                    'EurHigh_y','EurLow_y', 'EurClose_y',
                                    'EurWeekday_y', 'Open_unix_timestamp',
                                    'EurOpen_x', 'EurHigh_x','EurLow_x', 
                                    'EurClose_x','EurWeekday_x'
                                    ]
                           )

In [18]:
# Let's build a index selector for df_close_btc
close_unit_index = 0
value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
final_close_unit_index = df_open_btc.shape[0]
fruit_list = list(range(200))
serial_number_for_loop = 0
serial_number_if_loop = 0
serial_number_elif_loop_1 = 0
serial_number_elif_loop_2 = 0
serial_number_else_loop = 0
for x in fruit_list:#while close_unit_index < final_close_unit_index:
  serial_number_for_loop = serial_number_for_loop + 1
  print('For loop run number:',serial_number_for_loop)
  if value_close_units == 0:
    i = 1
  else:
    i = 0
  print('i =',i)
  close_unit_index = close_unit_index + i
  value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
  # How to set up an upper limit on this variable?? use df.shape[0]
  # Choosing first row from df_close_btc where the units are = values_of_units
  # Assign it as new DF
  # change the reset the index and the drop the 'index' column
  df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
  df_latest_row_from_close.reset_index(inplace=True)
  current_close_index = df_latest_row_from_close.at[0,'index']
  df_latest_row_from_close = df_latest_row_from_close.drop('index',axis=1)
  # Get the datetime value from the df_latest_row_from_close
  close_unit_datetime = df_close_btc.loc[close_unit_index,'Close Datetime']
  print('close_unit_datetime:',close_unit_datetime)
  # find all the transaction datetimes from df_open that were opened before the close_unit_datetime
  open_datetimes_b4 = df_open_btc['Open Datetime'][df_open_btc['Open Datetime'] < close_unit_datetime]
  #Latest datetime beofore close datetime
  open_datetime_lifo = open_datetimes_b4.max()
  print('open_datetime_lifo:',open_datetime_lifo)
  '''
  Remember that even if only one value for open_datetime_lifo is chosen
  this can still result in chosing more than one rows in that case last row 
  should be chosen
  '''
  # Corresponding row for the open_datetime_lifo 
  # let's make a new DF
  df_latest_last_row_open = df_open_btc[df_open_btc['Open Datetime'] == open_datetime_lifo]
  df_latest_last_row_open = df_latest_last_row_open[::-1]
  df_latest_last_row_open = df_latest_last_row_open[0:1]
  df_latest_last_row_open.reset_index(inplace=True)
  current_open_index = df_latest_last_row_open.at[0,'index']
  df_latest_last_row_open = df_latest_last_row_open.drop('index', axis=1)
  value_open_units = df_latest_last_row_open.loc[0,'Open Units']
  print('value_close_units:',value_close_units, 'value_open_units:',value_open_units)
  abs_diff_open_close_units = abs(value_close_units - value_open_units)
  print('abs_diff_open_close_units:',abs_diff_open_close_units)
  value_open_amount = df_latest_last_row_open.loc[0,'Open Amount_Eur']
  value_close_amount = df_latest_row_from_close.loc[0,'Amount_close_Eur']
  value_close_spread_eur = df_latest_row_from_close.loc[0,'Spread_Eur']
  close_Rate_eur = df_latest_row_from_close.loc[0,'Close Rate_Eur']
  open_Rate_eur = df_latest_last_row_open.loc[0,'Open Rate_Eur']
  #Remember that 'Open_index' and 'Close_index' columns host original indices
  # This is important as df_open_btc starts to drop rows the original indices will come in handy 
  # And in case of df_close_btc this should remain equal to the indices from DataFrame
  original_open_index = df_latest_last_row_open.at[0, 'Original_Open_index']
  #Remeber that earlier defined close_unit_index is the index of units from df_close_btc DataFrame
  # close_index should essentially be equal in integer value of close_unit_index
  close_index = df_latest_row_from_close.at[0, 'Original_Close_index']
  is_it_equal = close_unit_index == close_index
  print('(close_unit_index = close_index) is:',is_it_equal)
  if value_close_units > value_open_units:
    serial_number_if_loop = serial_number_if_loop + 1
    print('If_loop run number:',serial_number_if_loop)
    #First lets extract all the values from open DF df_latest_last_row_open
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
    df_open_btc = df_open_btc.drop(current_open_index,axis=0)
    #Then reset index and drop the new column 'index' in df_open_btc
    df_open_btc.reset_index(inplace=True)
    df_open_btc = df_open_btc.drop('index',axis=1)
    #Now lets work on the Close_side
    #First define the changed values for df_latest_row_from_close
    changed_value_close_units = value_open_units
    changed_value_close_amount = changed_value_close_units * close_Rate_eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    # Now lets extract all the values from df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('If_loop run number:',serial_number_if_loop,'finished')
  elif value_close_units < value_open_units:
    serial_number_elif_loop_1 = serial_number_elif_loop_1 + 1
    print('Elif_loop_1 run number:', serial_number_elif_loop_1)
    ## First lets extract all the values form close DF df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    '''
    #First define the changed values for df_latest_row_from_close
    ## In elif statement changed values are same as we made only one change
    changed_value_close_units = value_close_units # value_open_units in if statement was here change to value_close_unit
    changed_value_close_amount = changed_value_close_units * close_Rate_Eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    # and after that change the value of units and amount and spread in original df_close_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    '''
    # Now lets extract all the values from df_latest_row_from_close
    ## Now lets work on the values from open DF df_latest_last_row_lifo
    ## Remember we want to get back the values in the df_latest_row_from_close 
    ## we will have to assign changed_values for open_units and open_amount
    changed_value_open_units = value_close_units
    changed_value_open_amount = changed_value_open_units * open_Rate_eur
    ##After the values are changed extract the values which will then be reported
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    ## and then this time change the original values in the df_open_units
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_open_btc.at[current_open_index,'Open Units'] = value_open_units - changed_value_open_units
    df_open_btc.at[current_open_index,'Open Amount_Eur'] = value_open_amount - changed_value_open_amount
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_1 run number:', serial_number_elif_loop_1, 'finished')
  elif value_close_units == value_open_units:
    serial_number_elif_loop_2 = serial_number_elif_loop_2 + 1
    if value_close_units == 0:
      print('We are running value_close_units == 0 condition')
      print('This is in the Elif_loop_number_2:',serial_number_elif_loop_2)
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc = df_open_btc.drop(current_open_index,axis=0)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc = df_open_btc.drop('index',axis=1)
      print('We are breaking')
      break
    else:
      #First lets extract all the values from open DF df_latest_last_row_open
      newth_list_open_half = df_latest_last_row_open.to_dict('records')
      newth_dict_open_half = newth_list_open_half[0]
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc = df_open_btc.drop(current_open_index,axis=0)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc = df_open_btc.drop('index',axis=1)
      ### Second lets extract all the values form close DF df_latest_row_from_close
      newth_list_close_half = df_latest_row_from_close.to_dict('records')
      newth_dict_close_half = newth_list_close_half[0]
      #First define the changed values for df_latest_row_from_close
      changed_value_close_units = value_open_units
      changed_value_close_amount = changed_value_close_units * close_Rate_eur
      changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
      # and after that change the value of units and amount in original df_open_btc
      df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
      df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
      df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_2 run number:', serial_number_elif_loop_2, 'finished')
  else:
    serial_number_else_loop = serial_number_else_loop + 1
    print('Do we need to write more code? The answer is a: BIG YES!!')
    print('Or on second thought this is a silly mistake!!!')
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('serial_number_else_loop:', serial_number_else_loop, 'finished')
    print('We are breaking here!!')
    break
  print('For loop run number:',serial_number_for_loop,'finished')
  print('****************************************')


For loop run number: 1
i = 0
close_unit_datetime: 2020-05-01 11:44:24
open_datetime_lifo: 2020-04-30 19:14:44
value_close_units: 0.014149 value_open_units: 0.002861
abs_diff_open_close_units: 0.011288
(close_unit_index = close_index) is: True
If_loop run number: 1
value_close_units: 0.014149
value_open_units: 0.002861
close_unit_index: 0
If_loop run number: 1 finished
For loop run number: 1 finished
****************************************
For loop run number: 2
i = 0
close_unit_datetime: 2020-05-01 11:44:24
open_datetime_lifo: 2020-04-30 10:04:41
value_close_units: 0.011288 value_open_units: 0.002867
abs_diff_open_close_units: 0.008421
(close_unit_index = close_index) is: True
If_loop run number: 2
value_close_units: 0.011288
value_open_units: 0.002867
close_unit_index: 0
If_loop run number: 2 finished
For loop run number: 2 finished
****************************************
For loop run number: 3
i = 0
close_unit_datetime: 2020-05-01 11:44:24
open_datetime_lifo: 2020-04-30 04:14:49
va

In [ ]:
df_latest_row_from_close

In [ ]:
df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
df_latest_row_from_close

,Close_index,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,0,599996081,Buy Bitcoin,0.014149,2020-05-01 11:44:24,9027.0,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,115.940563,0.877881


In [ ]:
df_latest_last_row_open[-1:,:]

NameError: ignored

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import calendar
df_open = pd.read_csv('/content/et_open_alldata.csv')
df_close = pd.read_csv('/content/et_close_alldata.csv')
df_close['Close Datetime'] = pd.to_datetime(df_close['Close_unix_timestamp'],dayfirst=True,unit='s')
df_open['Open Datetime'] = pd.to_datetime(df_open['Open_unix_timestamp'],dayfirst=True,unit='s')
df_close_btc = df_close[df_close['Close Action'] =='Buy Bitcoin']
df_open_btc = df_open[df_open['Open Action'] =='Buy Bitcoin']
df_close_btc.reset_index(inplace=True)
df_close_btc = df_close_btc.drop('index', axis=1)
df_close_btc.reset_index(inplace=True)
df_close_btc.rename(columns={'index':'Original_Close_index'},inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc = df_open_btc.drop('index', axis=1)
df_open_btc.reset_index(inplace=True)
df_open_btc.rename(columns={'index':'Original_Open_index'},inplace=True)
data_btc = {}
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Original_Open_index','Original_Close_index',
                                    'Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units','Spread_Eur',
                                    'Open Amount_Eur','Amount_close_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur', 'Close Rate_Eur',
                                    'Open Action', 'Close Action', 
                                    'Open Amount','Close Rate', 'Open Rate',
                                    'Close Spread', 'Close Profit USD',
                                    'Close_Date','Open_Date',
                                    'Close_unix_timestamp','EurOpen_y', 
                                    'EurHigh_y','EurLow_y', 'EurClose_y',
                                    'EurWeekday_y', 'Open_unix_timestamp',
                                    'EurOpen_x', 'EurHigh_x','EurLow_x', 
                                    'EurClose_x','EurWeekday_x'
                                    ]
                           )
for x in fruit_list:#while close_unit_index < final_close_unit_index:
  serial_number_for_loop = serial_number_for_loop + 1
  print('For loop run number:',serial_number_for_loop)
  if value_close_units == 0:
    i = 1
  else:
    i = 0
  close_unit_index = close_unit_index + i
  value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
  print(value_close_units)
  # How to set up an upper limit on this variable??
  # Choosing first row from df_close where the units are = values_of_units
  # Assign it as new DF
  # change the reset the index and the drop the 'index' column
  df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
  df_latest_row_from_close.reset_index(inplace=True)
  df_latest_row_from_close.drop('index',axis=1,inplace=True)
  df_latest_row_from_close
  # Get the datetime value from the df_latest_row_from_close
  close_unit_datetime = df_close_btc.loc[close_unit_index,'Close Datetime']
  close_unit_datetime
  # find all the transaction datetimes from df_open that were opened before the close_unit_datetime
  open_datetimes_b4 = df_open_btc['Open Datetime'][df_open_btc['Open Datetime'] < close_unit_datetime]
  open_datetimes_b4
  #Latest datetime beofore close datetime
  open_datetime_lifo = open_datetimes_b4.max()
  open_datetime_lifo
  # Corresponding row for the open_datetime_lifo 
  # let's make a new DF
  df_latest_last_row_open = df_open_btc[df_open_btc['Open Datetime'] == open_datetime_lifo]
  df_latest_last_row_open.reset_index(inplace=True)
  df_latest_last_row_open.drop('index', axis=1,inplace=True)
  df_latest_last_row_open
  value_open_units = df_latest_last_row_open.loc[0,'Open Units']
  print(value_close_units, value_open_units)
  value_open_amount = df_latest_last_row_open.loc[0,'Open Amount_Eur']
  value_open_amount
  value_close_amount = df_latest_row_from_close.loc[0,'Amount_close_Eur']
  value_close_amount
  value_close_spread_eur = df_latest_row_from_close.loc[0,'Spread_Eur']
  value_close_spread_eur
  close_Rate_Eur = df_latest_row_from_close.loc[0,'Close Rate_Eur']
  close_Rate_Eur
  open_Rate_Eur = df_latest_last_row_open.loc[0,'Open Rate_Eur']
  open_Rate_Eur
  open_index = df_latest_last_row_open.at[0, 'Open_index']
  close_index = df_latest_row_from_close.at[0, 'Close_index']
  if value_close_units > value_open_units:
    serial_number_if_loop = serial_number_if_loop + 1
    print('If_loop run number:',serial_number_if_loop)
    #First lets extract all the values from open DF df_latest_last_row_open
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
    df_open_btc.drop(open_index,axis=0,inplace=True)
    #Then reset index and drop the new column 'index' in df_open_btc
    df_open_btc.reset_index(inplace=True)
    df_open_btc.drop('index',axis=1,inplace=True)
    #Now lets work on the Close_side
    #First define the changed values for df_latest_row_from_close
    changed_value_close_units = value_open_units
    changed_value_close_amount = changed_value_close_units * close_Rate_Eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    # Now lets extract all the values from df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    # Now A
    #df_latest_row_from_close = df_close_btc.loc[close_unit_index, 'Close Units']
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('If_loop run number:',serial_number_if_loop,'finished')
  elif value_close_units < value_open_units:
    serial_number_elif_loop_1 = serial_number_elif_loop_1 + 1
    print('Elif_loop_1 run number:', serial_number_elif_loop_1)
    ## First lets extract all the values form close DF df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    #First lets extract all the values from open DF df_latest_last_row_open
    #newth_list_open_half = df_latest_last_row_open.to_dict('records')
    #newth_dict_open_half = newth_list_open_half[0]
    #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
    #df_open_btc.drop(open_index,axis=0,inplace=True)
    #Then reset index and drop the new column 'index' in df_open_btc
    #df_open_btc.reset_index(inplace=True)
    #df_open_btc.drop('index',axis=1,inplace=True)
    #Now lets work on the Close_side
    #First define the changed values for df_latest_row_from_close
    ## In elif statement changed values are same as we made only one change
    changed_value_close_units = value_close_units # value_open_units in if statement was here change to value_close_unit
    changed_value_close_amount = changed_value_close_units * close_Rate_Eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    # and after that change the value of units and amount and spread in original df_close_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    # Now lets extract all the values from df_latest_row_from_close
    ## Now lets work on the values from open DF df_latest_last_row_lifo
    ## Remember we want to get back the values in the df_latest_row_from_close 
    ## we will have to assign changed_values for open_units and open_amount
    changed_value_open_units = value_close_units
    changed_value_open_amount = changed_value_open_units * open_Rate_Eur
    ##Theses changed values need to assigned to the current df_latest_last_row_open
    df_latest_last_row_open.at[0,'Open Units'] = changed_value_open_units
    df_latest_last_row_open.at[0,'Open Amount_Eur'] = changed_value_open_amount
    ##After the values are changed extract the values which will then be reported
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    ## and then this time change the original values in the df_open_units
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    iRowNumber_df_open_btc = df_latest_last_row_open.at[0,'Open_index']
    df_open_btc.at[iRowNumber_df_open_btc,'Open Units'] = value_open_units - changed_value_open_units
    df_open_btc.at[iRowNumber_df_open_btc,'Open Amount_Eur'] = value_open_amount - changed_value_open_amount
    #Then permanently delete the first row of the close DF df_latest_row_from_close from df_close_btc
    #df_close_btc.drop(close_index,axis=0,inplace=True)
    #Then reset index and drop the new column 'index' in df_open_btc
    #df_close_btc.reset_index(inplace=True)
    #df_close_btc.drop('index',axis=1,inplace=True)
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_1 run number:', serial_number_elif_loop_1, 'finished')
  elif value_close_units == value_open_units:
    serial_number_elif_loop_2 = serial_number_elif_loop_2 + 1
    print('Elif_loop_2 run number:', serial_number_elif_loop_2)
    if value_close_units == 0:
      #Then permanently delete the first row of the close DF df_latest_row_from_close from df_close_btc
      #df_close_btc.drop(close_index,axis=0,inplace=True)
      #Then reset index and drop the new column 'index' in df_open_btc
      #df_close_btc.reset_index(inplace=True)
      #df_close_btc.drop('index',axis=1,inplace=True)
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc.drop(open_index,axis=0,inplace=True)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc.drop('index',axis=1,inplace=True)
    else:
      #First lets extract all the values from open DF df_latest_last_row_open
      newth_list_open_half = df_latest_last_row_open.to_dict('records')
      newth_dict_open_half = newth_list_open_half[0]
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc.drop(open_index,axis=0,inplace=True)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc.drop('index',axis=1,inplace=True)
      ### Second lets extract all the values form close DF df_latest_row_from_close
      newth_list_close_half = df_latest_row_from_close.to_dict('records')
      newth_dict_close_half = newth_list_close_half[0]
      #First define the changed values for df_latest_row_from_close
      changed_value_close_units = value_open_units
      changed_value_close_amount = changed_value_close_units * close_Rate_Eur
      changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
      # and after that change the value of units and amount in original df_open_btc
      df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
      df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
      df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_2 run number:', serial_number_elif_loop_2, 'finished')
  else:
    serial_number_else_loop = serial_number_else_loop + 1
    print('Do we need to write more code? The answer is a: BIG YES!!')
    print('Or on second thought this is a silly mistake!!!')
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('serial_number_else_loop:', serial_number_else_loop, 'finished')
    break
    print('For loop run number:',serial_number_for_loop,'finished')
  print('***********************************')
